# 科学技術計算 5 レポート


In [2]:
import numpy as np
import scipy

from numpy.linalg import inv, norm, solve, det, matrix_rank, cond, pinv, lstsq, eig, eigh, eigvals, eigvalsh
from numpy import diag
rng = np.random.default_rng()

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams["savefig.bbox"] = "tight"

from typing import List, Optional, Union, Tuple

import sklearn
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces

from matplotlib.pyplot import imshow
from typing import Tuple, List, Optional, Union
from tqdm.auto import tqdm



## 課題05-1

In [4]:
#レイリー商反復について

#関数
def rayleigh_quotient_iteration(A: np.ndarray,mu: float, u, tol = 1e-12,n = np.ndarray, maxiter: int = 5000,) -> Tuple[float, np.ndarray]:
    """Rayleigh quotioent iteration for computing the largest eigenvalue and its eigenvector of A

    Args:
        A (np.ndarray): nxn matrix A
        mu (float): initial eigenvalue
        u (np.ndarray): n-d initial eigenvector
        maxiter (int, optional): max iterations. Defaults to 50.

    Returns:
        Tuple[float, np.ndarray]: the largest eigenvalue and its eigenvector
    """
    #入力 行列Aが対称かどうか muがシフト値uは
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    assert len(u) == n

    u_pre = u.copy()

    #レイリー商反復
    for i in range(maxiter):

        try:
            u = solve(A - mu * np.eye(n), u)    #シフト値muを用いて連立方程式を解く
        except:
            break
        
        u /= norm(u)    #正規化
        mu = u.T @ A @ u

        with np.printoptions(formatter={'float': '{: 12.12f}'.format}, suppress=True, linewidth=100):
            print(f"{i:2d}", u)

        if np.allclose(u, u_pre) or np.allclose(u, -u_pre):         #uの値がめっちゃちかくなった→近似
            break
        u_pre = u.copy()
        
    return mu, u



In [210]:
#ハウスホルダー法
def householder_step( A: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    """k-th step of Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix
        k (int): step

    Returns:
        Tuple[np.ndarray, np.ndarray]: transformed matrix QAQ after k-th step, and the transfrom Q
    """
    assert A.ndim == 2  #2次元であるかチェック
    assert A.shape[0] == A.shape[1] #正方行列かどうかチェック

    n = A.shape[0]
    assert 0 <= k < n - 2

    x = A[k + 1:, k]  # Aのk列目のk+1行目以降の部分を取得(縦の部分)
    y = np.zeros_like(x) #全部ゼロの配列
    # y[0] = - np.sign(A[k + 1, k]) * norm(x)
    y[0] = - np.sign(x[0]) * norm(x)  # ベクトルyの先頭要素を設定

    # print("x", x)
    # print("y", y)

    u = (x - y) / norm(x - y)  # 単位ベクトルuを計算
    H = np.eye(len(x)) - 2 * np.outer(u, u)  # ハウスホルダー行列Hを計算

    Q = np.eye(n)   #単位行列
    Q[k + 1:, k + 1:] = H  # QにHを組み込む

    QAQ = Q @ A @ Q  # 変換後の行列を計算
    return QAQ, Q

def householder_transform(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """Householder transform from nxn symmetric to tridiagonal

    Args:
        A (np.ndarray): an nxn symmetric matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: tridiagonal Atri and orthogonal Qhh
                                       so that Qhh A Qhh == Atri
    """
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    Atri = A.copy()
    Qhh = np.eye(n)
    for k in range(n - 2):
        # print(f"{k}-th step")
        Atri, Q = householder_step(Atri, k)
        Qhh = Qhh @ Q  # 直交行列を更新
        # print("Atri\n", Atri, sep='')
        # print("Q\n", Q, sep='')
        # print("Qhh\n", Qhh, sep='')
    return Atri, Qhh


In [211]:
#ぎぶんす変換
def givens_r_matrix(xp_xq: np.ndarray) -> np.ndarray:
    """generate a matrix of Givens rotation

    Args:
        xp_xq (ndarray): two values xp, xq

    Returns:
        ndarray: 2D rotation matrix
    """
    assert len(xp_xq) == 2

    xp, xq = xp_xq[0], xp_xq[1]

    # denominator = np.sqrt(xp**2 + xq**2)
    denominator = norm(xp_xq)
    c = xp / denominator
    s = -xq / denominator

    R = np.array([
        [c, s],
        [-s, c]
    ])
    return R

def qr_decom_by_givens(Atri_org: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    """QR decomposition of a tridiagonal matrix by Givens rotation

    Args:
        Atri_org (np.ndarray): an nxn tridiagonal matrix

    Returns:
        Tuple[np.ndarray, np.ndarray]: orthogonal Q and upper traiangular R so that A == QR
    """
    assert len(Atri_org.shape) == 2
    assert Atri_org.shape[0] == Atri_org.shape[1]
    # assert AAtri_org should be tridiagonal
    n = Atri_org.shape[0]

    Atri = Atri_org.copy()
    Q = np.eye(n)

    for k in range(n - 1):
        p, q = k, k + 1
        Rot2x2 = givens_r_matrix(Atri[[p, q]][:, p])
        Atri[[p, q]] = Rot2x2.T @ Atri[[p, q]]
        Q[:, [p, q]] = Q[:, [p, q]] @ Rot2x2

    R = Atri
    return Q, R

In [212]:
#QR法関数
def eigh_by_qr_method(A0: np.ndarray, maxiter: int = 1000,) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computing eigenvalues and eigenvectors by QR decompoition

    Args:
        A0 (np.ndarray): nxn matrix
        maxiter (int, optional): max iterations. Defaults to 1000.
        qr_decomp (callable, optional): QR decomposition solver. Defaults to np.linalg.qr.

    Returns:
        Tuple[np.ndarray, np.ndarray]: n-d vector of eigenvalues and orthogonal matrix with eigenvector columns
    """
    assert A0.ndim == 2
    assert A0.shape[0] == A0.shape[1]
    n = A0.shape[0]
    A = A0.copy()

    Qall = np.eye(n)
    diagonals_pre = diag(A)
    #ハウスホルダー変換
    Atri, Qhh = householder_transform(A)
    Qall = Qhh
    Atri_org = Atri.copy()

    for i in range(maxiter):
        
        Q, R = qr_decom_by_givens(Atri_org)
        Atri_org = R @ Q
        Qall = Qall @ Q

        diagonals = diag(Atri_org)
        # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        #     print(f"     diagonals {i:2d}", diagonals)
        #     print(f"superdiagonals {i:2d}", superdiagonals)

        if (np.allclose(diagonals, diagonals_pre)):
            # with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
            #     print("A\n", A, sep='')
            break

        diagonals_pre = diagonals.copy()

    return diagonals, Qall


In [213]:
#今回のやつ
def eigh_via_qr_rayleigh( A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray]:
    # print("A =", A)
    assert A.ndim == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]

    
    eigenvalues_approx, Qqr  = eigh_by_qr_method(A) #これでAがぎぶんす変換で更新しながら固有値と固有べくとるを出力できたってことかな

    eigenvalues = np.zeros(n)
    eigenvectors = np.zeros((n, n))

    for k in range(n):
        mu0 = eigenvalues_approx[k]
        u0 = Qqr[k, :]


        mu_refined, u_refined = rayleigh_quotient_iteration(A, mu0, u0)

        eigenvalues[k] = mu_refined
        eigenvectors[:, k] = u_refined

    return eigenvalues, eigenvectors


In [214]:
#main
#実行
for i in tqdm(range(10)):
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(n, n))

    #対称行列
    A = A.T @ A
    print("行列 A:")
    print(A)

    d, U = eigh_via_qr_rayleigh(A)
    eigvals_np, eigvecs_np = np.linalg.eigh(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        d_np, U_np = np.linalg.eigh(A)
        U_np = U_np[:, ::-1]
        d_np = d_np[::-1]

        # print("d_np", d_np)
        # print("d   ", d)
        print("d == d_np", np.allclose(d, d_np))

        # print("U_np\n", U_np, sep='')
        # print("U\n", U, sep='')

        for i in range(len(U)):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )



  0%|          | 0/10 [00:00<?, ?it/s]

行列 A:
[[2.39987935 1.35218589 1.08609952 2.08776144 1.74265562 1.56053878]
 [1.35218589 1.22199847 0.96185957 1.20092069 1.50667139 1.15093259]
 [1.08609952 0.96185957 1.09304196 1.08595342 1.51343625 1.39915741]
 [2.08776144 1.20092069 1.08595342 2.26833912 1.70420652 2.04191779]
 [1.74265562 1.50667139 1.51343625 1.70420652 2.68238923 1.95119801]
 [1.56053878 1.15093259 1.39915741 2.04191779 1.95119801 2.49511458]]
 0 [-0.435462372235 -0.309792264033 -0.300277602459 -0.445452413591 -0.470099215421 -0.452563268351]
 1 [ 0.435462372235  0.309792264033  0.300277602459  0.445452413591  0.470099215421  0.452563268351]
 0 [ 0.594756529100 -0.079538933015 -0.326779472604  0.454202995454 -0.540324771254 -0.186821267532]
 1 [ 0.594746177557 -0.079546314407 -0.326781624707  0.454211081021 -0.540332660597 -0.186804838452]
 2 [-0.594746177557  0.079546314407  0.326781624707 -0.454211081021  0.540332660597  0.186804838452]
 0 [-0.367816584888 -0.394567487045 -0.013028175872  0.284210258118 -0.346

## 課題05-3

In [152]:
#QR分解を利用する関数 full/reduced QR分解を利用したthin SVDの計算方法
def svd_via_qr(A: np.ndarray,) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    #行列のサイズ計算
    m, n = A.shape
    print("m =",m)
    print("n =",n)

    if(m >= n):
        print("nyaaaa1")
        #A.T @ A(対称行列)の対角化→固有値計算→QR文分解
        ATA = A.T @ A
        sigma, V = np.linalg.eigh(ATA)
        # print("V =", V)
        sigma = np.sqrt(np.maximum(sigma, 0))
        #V = n.n直交行列、sigma 対角行列
        V = V[:,::-1]
        sigma = sigma[::-1]
        # print("V =", V)
        a = A @ V
        U, c = np.linalg.qr(a)
        # U, c = scipy.linalg.qr(a)


    elif(m < n):
        print("nyaaaa2")
        AAT = A @ A.T
        sigma , U =  np.linalg.eigh(AAT)
        sigma = np.sqrt(np.maximum(sigma, 0))
        U = U[:,::-1]
        sigma = sigma[::-1]
        # print("U = ", U)
        a = A.T @ U
        # print("a =",a)
        V ,c = np.linalg.qr(a)
        print("V = ",V)
        # V, c = scipy.linalg.qr(a)


    VT = V.T


    return U, sigma, VT

In [172]:
#固有値分解を利用した単純なthinSVDの計算方法
def svd_via_eigh( A: np.ndarray, ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    #行列のサイズ計算
    m, n = A.shape
    print("m =",m)
    print("n =",n)

    if(m >= n):
        print("nyaaaa1")
        #A.T @ A(対称行列)の対角化→固有値計算→QR文分解
        ATA = A.T @ A
        sigma, V = np.linalg.eigh(ATA)
        V = V[:,::-1] 
        sigma = np.sqrt(np.maximum(sigma, 0))
        sigma = sigma [::-1]
        Sigma_inv = np.diag(1.0 / sigma) # 各対角成分を逆数にするだけ
        # print("V =", V)
        print("sigma =",sigma)
        print("Sigma_inv =", Sigma_inv)
        
        U = A @ V @ Sigma_inv
    elif(m < n):
        print("nyaaaa2")
        AAT = A @ A.T
        sigma , U =  np.linalg.eigh(AAT)
        sigma = np.sqrt(np.maximum(sigma, 0))
        sigma = sigma [::-1]
        U = U[:,::-1]
        Sigma_inv = np.diag(1.0 / sigma) # 各対角成分を逆数にするだけ
        Sigma_inv = np.diag(1.0 / sigma)
        V = A.T @ U @ Sigma_inv
    VT = V.T
    return U ,sigma, VT

In [179]:
#実行
# A = np.array([[1., 2., 3., 4.],
#               [4., 5., 6., 4.],
#               [7., 8., 9.,3.],
#                [10., 11., 12.,2.],
#                 [12.,12.,1.,12],
#                  [1.,2.,3.,12] ])

# A = np.array([[1., 2., 3., 4.,6.],
#               [5., 6., 7., 8.,5.],
#               [9., 10., 11., 1.,4.],
#               [1.,2.,2.,2.,2.]])


for i in tqdm(range(10)):
    m = rng.integers(low = 2, high = 10)
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(m, n))

    U, sigma, VT = svd_via_qr(A)


    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        U_np, sigma_np, VT_np = np.linalg.svd(A)
        
        print("sigma == sigma_np", np.allclose(sigma, sigma_np))

        for i in range(U.shape[1]):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )
        for i in range(len(VT)):
            print(
                f"column {i}: VT[{i}] == VT_np[{i}] ?",
                np.allclose(VT[i, :], VT_np[i, :]) or np.allclose(VT[i, :], -VT_np[i, :])
            )


for i in tqdm(range(10)):
    m = rng.integers(low = 2, high = 10)
    n = rng.integers(low = 2, high = 10)
    A = rng.random(size=(m, n))

    U, sigma, VT = svd_via_eigh(A)
        

    with np.printoptions(formatter={'float': '{: 12.8f}'.format}, suppress=True, linewidth=100):
        U_np, sigma_np, VT_np = np.linalg.svd(A)
        
        print("sigma == sigma_np", np.allclose(sigma, sigma_np))

        for i in range(U.shape[1]):
            print(
                f"column {i}: U[{i}] == U_np[{i}] ?",
                np.allclose(U[:, i], U_np[:, i]) or np.allclose(U[:, i], -U_np[:, i])
            )
        for i in range(len(VT)):
            print(
                f"column {i}: VT[{i}] == VT_np[{i}] ?",
                np.allclose(VT[i, :], VT_np[i, :]) or np.allclose(VT[i, :], -VT_np[i, :])
            )


  0%|          | 0/10 [00:00<?, ?it/s]

m = 6
n = 4
nyaaaa1
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 3: U[3] == U_np[3] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
column 2: VT[2] == VT_np[2] ? True
column 3: VT[3] == VT_np[3] ? True
m = 3
n = 5
nyaaaa2
V =  [[-0.42185064  0.65106621  0.60507159]
 [-0.52338143 -0.62838047  0.16009327]
 [-0.35663534  0.39988295 -0.74572316]
 [-0.55552648 -0.1439129   0.00615713]
 [-0.33513502 -0.02516894 -0.22829275]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
column 2: VT[2] == VT_np[2] ? True
m = 6
n = 6
nyaaaa1
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 3: U[3] == U_np[3] ? True
column 4: U[4] == U_np[4] ? True
column 5: U[5] == U_np[5] ? T

  0%|          | 0/10 [00:00<?, ?it/s]

m = 4
n = 4
nyaaaa1
sigma = [2.27309371 0.67822896 0.35767095 0.12479266]
Sigma_inv = [[0.43992907 0.         0.         0.        ]
 [0.         1.47442833 0.         0.        ]
 [0.         0.         2.79586585 0.        ]
 [0.         0.         0.         8.0132918 ]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 2: U[2] == U_np[2] ? True
column 3: U[3] == U_np[3] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
column 2: VT[2] == VT_np[2] ? True
column 3: VT[3] == VT_np[3] ? True
m = 5
n = 2
nyaaaa1
sigma = [1.77278078 0.17149696]
Sigma_inv = [[0.56408554 0.        ]
 [0.         5.83100728]]
sigma == sigma_np True
column 0: U[0] == U_np[0] ? True
column 1: U[1] == U_np[1] ? True
column 0: VT[0] == VT_np[0] ? True
column 1: VT[1] == VT_np[1] ? True
m = 9
n = 2
nyaaaa1
sigma = [2.48304389 0.99109767]
Sigma_inv = [[0.4027315  0.        ]
 [0.         1.00898229]]
sigma == sigma_np True
column 0: U[0] == 